**BASADO EN : https://www.kaggle.com/code/rishpande/pima-indians-diabetes-beginner/notebook **


# PIMA Indians Diabetes


## CONTEXTO

**Diabetes**, es un grupo de trastornos metabólicos en los que hay niveles elevados de azúcar en la sangre durante un período prolongado. Los síntomas de niveles altos de azúcar en la sangre incluyen micción frecuente, aumento de la sed y aumento del hambre. Si no se trata, la diabetes puede causar muchas complicaciones. Las complicaciones agudas pueden incluir cetoacidosis diabética, estado hiperglucémico hiperosmolar o muerte. Las complicaciones graves a largo plazo incluyen enfermedad cardiovascular, accidente cerebrovascular, enfermedad renal crónica, úlceras en los pies y daños en los ojos.
.

Este **dataset** es originario del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. El objetivo del conjunto de datos es predecir de manera diagnóstica si un paciente tiene o no diabetes, en función de ciertas medidas de diagnóstico incluidas en el conjunto de datos. Se impusieron varias restricciones a la selección de estas instancias de una base de datos más grande. En particular, todos los pacientes aquí son mujeres de al menos 21 años de herencia indígena pima.

## Objetivo

Construiremos un modelo de aprendizaje automático para predecir con precisión si los pacientes en el conjunto de datos tienen diabetes o no.

## Data


Los conjuntos de datos consisten en varias variables predictoras médicas y una variable objetivo, Resultado. Las variables predictoras incluyen el número de embarazos que ha tenido la paciente, su IMC, nivel de insulina, edad, etc.

* **Pregnancies**: Number of times pregnant
* **Glucose**: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
* **BloodPressure**: Diastolic blood pressure (mm Hg)
* **SkinThickness**: Triceps skin fold thickness (mm)
* **Insulin**: 2-Hour serum insulin (mu U/ml)
* **BMI**: Body mass index (weight in kg/(height in m)^2)
* **DiabetesPedigreeFunction**: Diabetes pedigree function
* **Age**: Age (years)
* **Outcome**: Class variable (0 or 1)




In [ ]:
!pip install plotly==3.10.0
from chart_studio import plotly

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
from IPython.display import HTML, Image


In [ ]:
df = pd.read_csv('sample_data/diabetes.csv')

In [ ]:
df.head(10)

In [ ]:
df.describe()

# Visualización de datos

****
**EJERCICIO:** a partir del siguiente gráfico identifique que tipo de modelo de clasificación podemos aplicar.
****

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (15, 7))
f.suptitle("Diabetes?", fontsize = 18.)
_ = df.Outcome.value_counts().plot.bar(ax = ax[0], rot = 0, color = (sns.color_palette()[0], sns.color_palette()[2])).set(xticklabels = ["No", "Yes"])
_ = df.Outcome.value_counts().plot.pie(labels = ("No", "Yes"), autopct = "%.2f%%", label = "", fontsize = 13., ax = ax[1],\
colors = (sns.color_palette()[0], sns.color_palette()[2]), wedgeprops = {"linewidth": 1.5, "edgecolor": "#F7F7F7"}), ax[1].texts[1].set_color("#F7F7F7"), ax[1].texts[3].set_color("#F7F7F7")

## Distribución


****
**EJERCICIO:** a partir de los siguientes gráficos, ¿qué puedes comentar del tipo de distribución de los datos?
****

In [ ]:
fig, ax = plt.subplots(4,2, figsize=(16,16))
sns.distplot(df.Age, bins = 20, ax=ax[0,0])
sns.distplot(df.Pregnancies, bins = 20, ax=ax[0,1])
sns.distplot(df.Glucose, bins = 20, ax=ax[1,0])
sns.distplot(df.BloodPressure, bins = 20, ax=ax[1,1])
sns.distplot(df.SkinThickness, bins = 20, ax=ax[2,0])
sns.distplot(df.Insulin, bins = 20, ax=ax[2,1])
sns.distplot(df.DiabetesPedigreeFunction, bins = 20, ax=ax[3,0])
sns.distplot(df.BMI, bins = 20, ax=ax[3,1])

In [ ]:
sns.regplot(x='SkinThickness', y= 'Insulin', data=df)

## Correlación



****
**EJERCICIO:** a partir de los siguientes gráficos, ¿qué puedes comentar de la correlación entre variables?.
****

In [ ]:
sns.pairplot(data=df,hue='Outcome')

## Gráficos de Violín



****
**EJERCICIO:** comenta los siguientes gráficos ¿qué muestran? ¿puedes identificar medidas de tendencia central?.
****

In [ ]:
fig,ax = plt.subplots(nrows=4, ncols=2, figsize=(18,18))
plt.suptitle('Violin Plots',fontsize=24)
sns.violinplot(x="Pregnancies", data=df,ax=ax[0,0],palette='Set3')
sns.violinplot(x="Glucose", data=df,ax=ax[0,1],palette='Set3')
sns.violinplot (x ='BloodPressure', data=df, ax=ax[1,0], palette='Set3')
sns.violinplot(x='SkinThickness', data=df, ax=ax[1,1],palette='Set3')
sns.violinplot(x='Insulin', data=df, ax=ax[2,0], palette='Set3')
sns.violinplot(x='BMI', data=df, ax=ax[2,1],palette='Set3')
sns.violinplot(x='DiabetesPedigreeFunction', data=df, ax=ax[3,0],palette='Set3')
sns.violinplot(x='Age', data=df, ax=ax[3,1],palette='Set3')
plt.show()


## Correlación entre características

****
**EJERCICIO:** comenta el siguiente mapa ¿qué correlaciones positivas existen? ¿cuáles son negativas?.
****

In [ ]:
corr=df.corr()

sns.set(font_scale=1.15)
plt.figure(figsize=(14, 10))

sns.heatmap(corr, vmax=.8, linewidths=0.01,
            square=True,annot=True,cmap='YlGnBu',linecolor="black")
plt.title('Correlation between features');

# Modelo predictivo

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import GradientBoostingClassifier

X = df.iloc[:, :-1]
y = df.iloc[:, -1]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

## Logistic Regression



****
**EJERCICIO:** identifica las etiquetas o clases que serán predichas por el modelo.
****

In [ ]:
#Model
LR = LogisticRegression()

#fiting the model
LR.fit(X_train, y_train)

#prediction
y_pred = LR.predict(X_test)



#Plot the confusion matrix
sns.set(font_scale=1.5)
cm = confusion_matrix(y_pred, y_test)
sns.heatmap(cm, annot=True, fmt='g')
plt.show()
#Accuracy
print()
print()
print("ACCURACY ", LR.score(X_test, y_test)*100)

****
**EJERCICIO:** comenta si este es un buen modelo, justifica. ¿Cómo está el balance de clases? ¿afecta al modelo?
****